In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


plt.close('all')
trainpath = 'nTrain'
testpath = 'Test'
tc = 3 #total number of  classes
Tc = np.arange(tc)
ic = 1 #number of impostor classes
c = tc - ic - 1 #last gallery class

cig = 20
cgi = 2
cgg = 1
cost = np.zeros([tc,tc])
DH = 15 #horizontal compression
DV = 15 #vertical compression
dim = (60,60) #image resize (nxm)

#cost matrix
for i in range(tc):
    for j in range(tc):
        if i==c+1 and j<=c:
            cost[i,j] = cig
        elif i<=c and j==c+1:
            cost[i,j] = cgi
        elif i<=c and j<=c and i!=j:
            cost[i,j] = cgg

M = [] # train image array
L = [] # train image label array
for file in os.listdir(trainpath):
    A = cv2.imread(trainpath+os.sep+file)
    A = cv2.resize(A[:,:,0], dim, interpolation = cv2.INTER_AREA)
    M.append(A)
    L.append(int(file[0]))

m= A.shape[0] #image original size
n = A.shape[1]    
M = np.array(M)
L = np.array(L)
#initialise scatter matrices
SHb = np.zeros([n,n])
SVb = np.zeros([m,m])
SHw = np.zeros([n,n])
SVw = np.zeros([m,m])

#Bstar calculation
for i in Tc:
    Ni = list(L==i).count(1)
    Ai_mean = np.mean(M[L==i], axis = 0)
    for j in Tc:
        Nj = list(L==j).count(1)
        Aj_mean = np.mean(M[L==j], axis = 0)
        if(i!=j):
            SHb += Ni*Nj*cost[i,j]*np.dot((Ai_mean - Aj_mean).T, Ai_mean - Aj_mean)
            SVb += Ni*Nj*cost[i,j]*np.dot(Ai_mean - Aj_mean, (Ai_mean - Aj_mean).T)
        
for i in Tc:
    f = np.sum(cost[i])
    Ai_mean = np.mean(M[L==i], axis = 0)
    for A in M[L==i]:   
        SHw += f*np.dot((A - Ai_mean).T, (A - Ai_mean))
#        print(i,f, Ni, SHw[0][0], Ai_mean[0][0])
        SVw += f*np.dot((A - Ai_mean), (A - Ai_mean).T)

tri = 'L'
uh = np.dot(np.linalg.inv(SHw), SHb)
uv = np.dot(np.linalg.inv(SVw), SVb)
[lambdauh, Uh] = np.linalg.eig(uh)
[lambdauv, Uv] = np.linalg.eig(uv)
for i in range(len(lambdauh)):
    eigv = Uh[:,i].reshape(Uh.shape[0],1)
    np.testing.assert_array_almost_equal(np.linalg.inv(SHw).dot(SHb).dot(eigv),
                                         lambdauh[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
print('ok')
lambdah = list(lambdauh) + list(lambdauv)
indicesh = np.argsort(lambdah)[::-1][0:DH]
h1 = indicesh[indicesh<len(lambdauh)]
h2 = indicesh[indicesh>=len(lambdauh)] - len(lambdauh)
   

Wh = Uh[:,h1]
Wv = Uv[:,h2]
Bh = [np.dot(M[i],Wh) for i in range(M.shape[0])]
Bv = [np.dot(M[i].T,Wv) for i in range(M.shape[0])]
Bh = np.array(Bh)
Bv = np.array(Bv)

#vstar computation
Shb = np.zeros([m,m])
Svb = np.zeros([n,n])
Shw = np.zeros([m,m])
Svw = np.zeros([n,n])
for i in Tc:
    Ni = list(L==i).count(1)
    BHi_mean = np.mean(Bh[L==i], axis = 0)
    BVi_mean = np.mean(Bv[L==i], axis = 0)
    for j in Tc:
        Nj = list(L==j).count(1)
        BHj_mean = np.mean(Bh[L==j], axis = 0)
        BVj_mean = np.mean(Bv[L==j], axis = 0)
        if(i!=j):
            Shb += Ni*Nj*cost[i,j]*np.dot(BHi_mean - BHj_mean, (BHi_mean - BHj_mean).T)
            Svb += Ni*Nj*cost[i,j]*np.dot((BVi_mean - BVj_mean), (BVi_mean - BVj_mean).T)
          

for i in Tc:
    f = np.sum(cost[i])
    BHi_mean = np.mean(Bh[L==i], axis = 0)
    BVi_mean = np.mean(Bv[L==i], axis = 0)
    for BH,BV in zip(Bh[L==i], Bv[L==i]):
          Shw += f*np.dot((BH - BHi_mean), (BH - BHi_mean).T)
          Svw += f*np.dot((BV - BVi_mean), (BV - BVi_mean).T)


vh = np.dot(np.linalg.inv(Shw), Shb)
vv = np.dot(np.linalg.inv(Svw), Svb)
[lambdavh, xh] = np.linalg.eig(vh)
[lambdavv, xv] = np.linalg.eig(vv)
lambdav = list(np.real(lambdavh)) + list(np.real(lambdavv))
indicesg = np.argsort(lambdav)[::-1][0:DV]
g1 = indicesg[indicesg<len(lambdavh)]
g2 = indicesg[indicesg>=len(lambdavh)] - len(lambdavh)
    

Vh = np.real(xh[:,g1])

Vv = np.real(xv[:,g2])
Vstar = np.block([[Vh, np.zeros([Vh.shape[0], len(g2)])],
                  [np.zeros([Vv.shape[0], len(g1)]), Vv]])
Wstar = np.block([[Wh, np.zeros([Wh.shape[0], len(h2)])],
                      [np.zeros([Wv.shape[0], len(h1)]), Wv]])

#Cstar feature
Astarimage = []
Cstarfeature = []
Bstarfeature = []
for i in range(M.shape[0]):
    A = M[i]
    Astar = np.block([[A, np.zeros([m,m])],
                          [np.zeros([n,n]), A.T]])
    Astarimage.append(Astar)
    Bstar = np.dot(Astar, Wstar)
    Bstarfeature.append(Bstar)
    Cstar = np.dot(Bstar.T, Vstar)
    Cstarfeature.append(Cstar.T)

Bstarfeature = np.array(Bstarfeature)
Astarimage = np.array(Astarimage)
Cstarfeature = np.array(Cstarfeature)

def feature(i,j):   
    pca = PCA(n_components=2)
    if(j == 0):
        x = StandardScaler().fit_transform(i)
        pca_r = pca.fit_transform(x)
        return pca_r
    if(j==1):
        x = StandardScaler().fit_transform(i)
        pca_r = pca.fit_transform(x)
        return pca_r
    
    
plt.figure()
plt.title('original images under pca')
imag = M[np.where(L==0)[0]]
imag_pca = np.empty([m*n,0])
s = 5
for I in imag:
    imag_pca = np.concatenate((imag_pca, I.flatten()[:,None]), axis = 1)
pca_r = feature(imag_pca, 0)
plt.scatter(pca_r[:,0], pca_r[:,1], c = 'blue', s= s)

imag = M[np.where(L==1)[0]]
imag_pca = np.empty([m*n,0])
for I in imag:
    imag_pca = np.concatenate((imag_pca, I.flatten()[:,None]), axis = 1)
pca_r = feature(imag_pca, 0)
plt.scatter(pca_r[:,0], pca_r[:,1], c = 'green', s= s)

imag = M[np.where(L==2)[0]]
imag_pca = np.empty([m*n,0])
for I in imag:
    imag_pca = np.concatenate((imag_pca, I.flatten()[:,None]), axis = 1)
pca_r = feature(imag_pca, 0)
plt.scatter(pca_r[:,0], pca_r[:,1], c = 'red', s= s)

plt.figure()
plt.title('cslda features under pca')
view = 5
imag = Cstarfeature[np.where(L==0)[0]]
imag_pca = np.empty([DH*DV,0])

for I in imag:
    imag_pca = np.concatenate((imag_pca, I.flatten()[:,None]), axis = 1)
    if(imag_pca.shape[1] == view):
        break
pca_r = feature(imag_pca, 1)
plt.scatter(pca_r[:,0], pca_r[:,1], c = 'blue', s= s)


imag = Cstarfeature[np.where(L==1)[0]]
imag_pca = np.empty([DH*DV,0])
for I in imag:
    imag_pca = np.concatenate((imag_pca, I.flatten()[:,None]), axis = 1)
    if(imag_pca.shape[1] == view):
        break
pca_r = feature(imag_pca, 1)
plt.scatter(pca_r[:,0], pca_r[:,1], c = 'green', s= s)

#
imag = Cstarfeature[np.where(L==2)[0]]
imag_pca = np.empty([DH*DV,0])
for I in imag:
    imag_pca = np.concatenate((imag_pca, I.flatten()[:,None]), axis = 1)
    if(imag_pca.shape[1] == view):
        break
pca_r = feature(imag_pca, 1)
plt.scatter(pca_r[:,0], pca_r[:,1], c = 'red', s= s)

plt.legend(labels = ['gallery1','gallery2','imposter'])

k = -6
plt.figure()
plt.title('Astar')
plt.imshow(Astarimage[k])

plt.figure()
plt.title('Wstar')
plt.imshow(Wstar)

plt.figure()
plt.title('Bstar')
plt.imshow(Bstarfeature[k])


plt.figure()
plt.title('Vstar.T')
plt.imshow(Vstar.T)

plt.figure()
plt.title('Cstar')
plt.imshow(Cstarfeature[k])
